https://www.youtube.com/watch?v=OX89LkTvNKQ&t=105s

https://github.com/langchain-ai/langchain-mcp-adapters

In [1]:
# MCP client
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# LangGraph agent 
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools

# dotenv
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())

In [2]:
# create execute LLM 
model = ChatOpenAI(
        model="gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_BASE_URL"),
        temperature=0.7)

# Server parameters
# - Handles the incoming connection(in this case, via stdio)
# - Handles tools discovery(when client ask "what tools are available")
# - Handles tool invocation(when client ask to use a tool)
# - Executes the tool call and sends the result back to the client
server_params = StdioServerParameters(
    command="python",
    args=["math_server.py"],
)


# 
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Convert MCP Tools to LangChain Tools 
        tools = await load_mcp_tools(session)
        
        # Create the agent and run 
        agent = create_react_agent(model, tools)
        agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})

In [4]:
for m in agent_response['messages']:
    m.pretty_print()

================================ Human Message =================================

what's (3 + 5) x 12?
================================== Ai Message ==================================
Tool Calls:
  add (call_J66eiLANJ2M619Qj9GuRHQ27)
 Call ID: call_J66eiLANJ2M619Qj9GuRHQ27
  Args:
    a: 3
    b: 5
  multiply (call_e2CQ4kzTsm2wF5Nhiz5Q6jgh)
 Call ID: call_e2CQ4kzTsm2wF5Nhiz5Q6jgh
  Args:
    a: 8
    b: 12
================================= Tool Message =================================
Name: add

8
================================= Tool Message =================================
Name: multiply

96
================================== Ai Message ==================================

The result of \((3 + 5) \times 12\) is 96.


In [12]:
from langchain_mcp_adapters.client import MultiServerMCPClient 

async with MultiServerMCPClient() as client:
    await client.connect_to_server(
        "math",
        command="python",
        # Make sure to update to the full absolute path to your math_server.py file
        args=["math_server.py"],
    )
    await client.connect_to_server(
        "weather",
        command="python",
        # Make sure to update to the full absolute path to your weather_server.py file
        args=["weather_server.py"],
    )
    agent = create_react_agent(model, client.get_tools())
    math_response = await agent.ainvoke({"messages": "what's (3 + 2) x 102?"})
    weather_response = await agent.ainvoke({"messages": "what is the weather in Hangzhou?"})

In [13]:
for m in math_response['messages']:
    m.pretty_print()

================================ Human Message =================================

what's (3 + 2) x 102?
================================== Ai Message ==================================
Tool Calls:
  add (call_mFWUNjAYnqHT7JdoXiK1navt)
 Call ID: call_mFWUNjAYnqHT7JdoXiK1navt
  Args:
    a: 3
    b: 2
  multiply (call_fcbGxPFNXOnJmZeL7ou7dvKm)
 Call ID: call_fcbGxPFNXOnJmZeL7ou7dvKm
  Args:
    a: 5
    b: 102
================================= Tool Message =================================
Name: add

5
================================= Tool Message =================================
Name: multiply

510
================================== Ai Message ==================================

(3 + 2) x 102 = 510


In [14]:
for m in weather_response['messages']:
    m.pretty_print()

================================ Human Message =================================

what is the weather in Hangzhou?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_tEisd1A1uemb2uAwnM1SGB1d)
 Call ID: call_tEisd1A1uemb2uAwnM1SGB1d
  Args:
    location: Hangzhou
================================= Tool Message =================================
Name: get_weather

{"basic": {"description": "clear sky", "temp": 9.95, "temp_min": 9.95, "temp_max": 9.95, "unit": "celsius"}, "advanced": {"feels_like": 9.1, "humidity": 36, "wind_speed": 2.04, "cloud_cover": 2, "pressure": 1034, "visibility": 10000, "sunrise": "06:31", "sunset": "17:54"}}
================================== Ai Message ==================================

The current weather in Hangzhou is as follows:
- Description: Clear sky
- Temperature: 9.95°C
- Feels Like: 9.1°C
- Humidity: 36%
- Wind Speed: 2.04 m/s
- Cloud Cover: 2%
- Pressure: 1034 hPa
- Visibility: 10 km
- Sunri